In [1]:
%matplotlib inline
import keras
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Add, concatenate , Subtract, Activation , average,multiply,add
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Dropout, BatchNormalization,Lambda
from keras.models import Model
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.dummy import DummyClassifier
import json
import numpy as np
np.random.seed(44)
from sklearn.utils import shuffle
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()

config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))
import math as mth
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
import scipy.spatial as sp
import pandas as pd
from pycm import *

Using TensorFlow backend.


In [2]:
link_data='../Data/data_multi_class.csv'


In [3]:
df_all=pd.read_csv(link_data, index_col=[0])

In [4]:
df_all.head()

,w1,w2,CAT_COH,CAT_HYPER,CAT_MERO,CAT_RANDOM,CAT_SYN
0,sensor,key,1,0,0,0,0
1,destructiveness,centaur,0,0,0,1,0
2,disenfranchisement,management,0,1,0,0,0
3,television,fridge,1,0,0,0,0
4,type,case,0,0,0,0,1


In [5]:

def load_embeddings(path, dimension):
    f = open(path, encoding="utf8").read().splitlines()
    vectors = {}
    for i in f:
        elems = i.split()
        vectors[" ".join(elems[:-dimension])] =  np.array(elems[-dimension:]).astype(float)
    return vectors

In [6]:
embeddings = load_embeddings("../Data/Embeddings/glove.6B.300d.txt", 300)

In [7]:
words_ = sorted(list(set(df_all.w1.values.tolist() + df_all.w2.values.tolist())))

words_train, words_test =train_test_split(words_, test_size=0.4)
   
df_all["known_words"] = df_all.apply(lambda l: l["w1"] in embeddings and l["w2"] in embeddings, axis =1  )
    
    
    
    
df_all["is_train"] = df_all.apply(lambda l : l["w1"] in words_train and l["w2"] in words_train and l["known_words"] == True, axis=1 )
df_all["is_test"] = df_all.apply(lambda l : l["w1"] in words_test and l["w2"] in words_test and l["known_words"] == True, axis=1)
 

In [8]:
df_all.head()

,w1,w2,CAT_COH,CAT_HYPER,CAT_MERO,CAT_RANDOM,CAT_SYN,known_words,is_train,is_test
0,sensor,key,1,0,0,0,0,True,False,False
1,destructiveness,centaur,0,0,0,1,0,True,True,False
2,disenfranchisement,management,0,1,0,0,0,True,False,False
3,television,fridge,1,0,0,0,0,True,False,False
4,type,case,0,0,0,0,1,True,False,False


In [9]:
def get_vector_representation_of_word_pairs(dataframe, embeddings_voca):
    x1 = [embeddings_voca[word] for word in dataframe.w1.values]
    x2 =[embeddings_voca[word] for word in dataframe.w2.values]
    y = dataframe[['CAT_COH','CAT_HYPER','CAT_MERO','CAT_RANDOM','CAT_SYN']].values
    x = np.hstack((x1, x2))
    

    return x, y

In [10]:
def get_vector_representation_of_word_pairs_multi(dataframe, embeddings_voca):
    x1 = [embeddings_voca[word] for word in dataframe.w1.values]
    x2 =[embeddings_voca[word] for word in dataframe.w2.values]
    y = dataframe.Category.values
    x = np.hstack((x1, x2))
    

    return x, y

In [11]:
len(df_all)

52699

In [12]:
xtrain, ytrain = get_vector_representation_of_word_pairs(df_all.loc[df_all.is_train==True], embeddings)
xtest, ytest   = get_vector_representation_of_word_pairs(df_all.loc[df_all.is_test==True], embeddings)

In [13]:
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, stratify=ytrain,  test_size=0.30, random_state=1234,)

In [14]:
len(xtrain),len(xvalid)

(13517, 5793)

In [15]:
len(xtest)

7868

In [16]:
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [72]:
from sklearn.linear_model import LogisticRegression

In [79]:
len(xtrain)

13517

In [88]:
x_train=xtrain[0:7000]
y_train=ytrain[0:7000]

In [89]:
classif = OneVsRestClassifier(SVC(kernel='linear'))
classif.fit(x_train, y_train)

OneVsRestClassifier(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
          n_jobs=None)

In [90]:
ypredict_test=classif.predict(xtest)

In [96]:
y_test=[list_cat[np.argmax(l)] for l in ytest]

In [92]:
df_Baseline=create_metrics(ytest,ypredict_test)

In [109]:
est=[list_cat[np.argmax(l)] for l in ypredict_test]

In [110]:
df_Baseline=create_metrics(ytest,est)

In [111]:
df_Baseline

,F1,ACC
COH,0.213078,0.645145
HYPER,0.354733,0.773894
MERO,0.449184,0.944204
RANDOM,0.588517,0.606507
SYN,0.048649,0.865785


In [107]:
df_Baseline

,F1,ACC
0,0.0,0.635231
1,0.0,0.878495
2,0.0,0.956406
3,0.0,0.541688
4,0.0,0.988180
COH,0.0,0.913828
HYPER,0.0,0.771098
MERO,0.0,0.942298
RANDOM,0.0,0.502034
SYN,0.0,0.870742


In [42]:
def create_metrics(y_test,y_pred):
    y_test=[list_cat[np.argmax(l)] for l in y_test]
    cm = ConfusionMatrix(actual_vector=y_test, predict_vector=y_pred) # Create CM From Data
    dict_F1=cm.class_stat['F1']
    df_metrics=pd.DataFrame.from_dict(dict_F1, orient='index',columns=['F1'])
    df_metrics['ACC']=pd.DataFrame.from_dict(cm.class_stat['ACC'], orient='index' )
    print(cm)

    #cm.class_stat
    return df_metrics

In [95]:
list_cat

['COH', 'HYPER', 'MERO', 'RANDOM', 'SYN']

In [93]:
df_Baseline

,F1,ACC
COH,0.0,0.913828
HYPER,0.0,0.771098
MERO,0.0,0.942298
RANDOM,0.0,0.502034
SYN,0.0,0.870742
"[0, 0, 0, 0, 0]",0.0,0.702339
"[0, 0, 0, 0, 1]",0.0,0.988180
"[0, 0, 0, 1, 0]",0.0,0.547153
"[0, 0, 0, 1, 1]",0.0,0.994535
"[0, 0, 1, 0, 0]",0.0,0.968353


In [19]:
#ypredict_test

In [18]:
def NN_Baseline_Multi():
    """Defines the NN baseline.
    Two hidden layers, followed by the output layer. 
    """
    model = Sequential()
    model.add(Dense(50, activation='sigmoid', input_dim=600))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    return model

In [19]:
def NN_Baseline_Multi_Task():
    inputs = Input(shape=(600,))
    
    x_shared=Dense(300, activation='sigmoid')(inputs)
    
    ind_coh=concatenate([inputs,x_shared])
    ind_hyper=concatenate([inputs,x_shared])
    ind_mero=concatenate([inputs,x_shared])
    ind_random=concatenate([inputs,x_shared])
    ind_syn=concatenate([inputs,x_shared])
    
    x_coh=Dense(50, activation='sigmoid')(ind_coh)
    x_hyper=Dense(50, activation='sigmoid')(ind_hyper)
    x_mero=Dense(50, activation='sigmoid')(ind_mero)
    x_random=Dense(50, activation='sigmoid')(ind_random)
    x_syn=Dense(50, activation='sigmoid')(ind_syn)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    out_syn=Dense(1, activation='sigmoid')(x_syn)
    
    out_pre=concatenate([out_coh,out_hyper,out_mero,out_random,out_syn])
    #out_final=Dense(5,activation='softmax')(out_pre)
    out_final=out_pre
    model = Model(inputs=[inputs], outputs=[out_final])
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    return model

In [21]:
def NN_Baseline_oneVsRest():
    #l'apprentissage de chaque partie est independant, c'est du vrai multi task
    inputs = Input(shape=(600,))
    

    
    x_coh=Dense(50, activation='sigmoid')(inputs)
    x_hyper=Dense(50, activation='sigmoid')(inputs)
    x_mero=Dense(50, activation='sigmoid')(inputs)
    x_random=Dense(50, activation='sigmoid')(inputs)
    x_syn=Dense(50, activation='sigmoid')(inputs)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    out_syn=Dense(1, activation='sigmoid')(x_syn)
    

    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs], outputs=[out_coh,out_hyper,out_mero,out_random,out_syn])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f])
    return model

In [55]:
def NN_Baseline_Multi_Task_indep():
    #l'apprentissage de chaque partie est independant, c'est du vrai multi task
    inputs = Input(shape=(600,))
    
    x_shared1=Dense(300, activation='sigmoid')(inputs)
    x_shared2=Dense(300, activation='sigmoid')(inputs)
    
  
    

    #x_random=Dense(50, activation='sigmoid')(ind_random)
    
    
    
    
    x_random=Dense(50, activation='sigmoid')(inputs)
    out_random=Dense(1, activation='sigmoid')(x_random)
    
    ind_coh=concatenate([inputs,x_shared1,out_random])
    ind_hyper=concatenate([inputs,x_shared1,out_random])
    ind_mero=concatenate([inputs,x_shared1,out_random])
    #ind_random=concatenate([inputs,x_shared2])
    ind_syn=concatenate([inputs,x_shared1,out_random])
    
    x_coh=Dense(50, activation='sigmoid')(ind_coh)
    x_hyper=Dense(50, activation='sigmoid')(ind_hyper)
    x_mero=Dense(50, activation='sigmoid')(ind_mero)
    x_syn=Dense(50, activation='sigmoid')(ind_syn)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    #out_random=Dense(1, activation='sigmoid')(x_random)
    out_syn=Dense(1, activation='sigmoid')(x_syn)
    

    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs], outputs=[out_coh,out_hyper,out_mero,out_random,out_syn])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f])
    return model

In [62]:
def NN_Baseline_Multi_Task_indep_adv():
    #l'apprentissage de chaque partie est independant, c'est du vrai multi task
    inputs = Input(shape=(600,))
    
    x_shared1=Dense(300, activation='sigmoid')(inputs)
    x_shared2=Dense(300, activation='sigmoid')(inputs)
    
  
    

    #x_random=Dense(50, activation='sigmoid')(ind_random)
    
    
    
    
    
    
    
    ind_coh=concatenate([inputs,x_shared1])
    ind_hyper=concatenate([inputs,x_shared1])
    ind_mero=concatenate([inputs,x_shared1])
    ind_random=concatenate([inputs,x_shared1])
    ind_syn=concatenate([inputs,x_shared1])
    
    x_coh=Dense(50, activation='sigmoid')(ind_coh)
    x_hyper=Dense(50, activation='sigmoid')(ind_hyper)
    x_mero=Dense(50, activation='sigmoid')(ind_mero)
    x_random=Dense(50, activation='sigmoid')(ind_random)
    x_syn=Dense(50, activation='sigmoid')(ind_syn)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    out_syn=Dense(1, activation='sigmoid')(x_syn)
    
    out_final=Dense(5, activation='softmax')(x_shared1)

    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs], outputs=[out_coh,out_hyper,out_mero,out_random,out_syn,out_final])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f,'categorical_crossentropy'])
    return model

In [135]:
def NN_Baseline_Multi_Task_indep_new_idea():
    #l'apprentissage de chaque partie est independant, c'est du vrai multi task
    inputs = Input(shape=(600,))
    
    x_shared=Dense(300, activation='sigmoid')(inputs)
    
    ind_coh=concatenate([inputs,x_shared])
    ind_hyper=concatenate([inputs,x_shared])
    ind_mero=concatenate([inputs,x_shared])
    ind_random=concatenate([inputs,x_shared])
    ind_syn=concatenate([inputs,x_shared])
    
    x_coh=Dense(50, activation='sigmoid')(ind_coh)
    x_hyper=Dense(50, activation='sigmoid')(ind_hyper)
    x_mero=Dense(50, activation='sigmoid')(ind_mero)
    x_random=Dense(50, activation='sigmoid')(ind_random)
    x_syn=Dense(50, activation='sigmoid')(ind_syn)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    out_syn=Dense(1, activation='sigmoid')(x_syn)
    
    #out_pre=concatenate([x_coh,x_hyper,x_mero,x_random,x_syn])
    out_pre=concatenate([out_coh,out_hyper,out_mero,out_random,out_syn])
    out_final=Dense(5,activation='softmax')(out_pre)

    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs], outputs=[out_coh,out_hyper,out_mero,out_random,out_syn,out_final])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f,'categorical_crossentropy'])
    return model

In [138]:
def NN_Baseline_Multi_Task_indep_new_idea2a2():
    #l'apprentissage de chaque partie est independant, c'est du vrai multi task
    inputs = Input(shape=(600,))
    
    x_shared=Dense(300, activation='sigmoid')(inputs)
    
    ind_coh=concatenate([inputs,x_shared])
    ind_hyper=concatenate([inputs,x_shared])
    ind_mero=concatenate([inputs,x_shared])
    ind_random=concatenate([inputs,x_shared])
    ind_syn=concatenate([inputs,x_shared])
    
    x_coh=Dense(50, activation='sigmoid')(ind_coh)
    x_hyper=Dense(50, activation='sigmoid')(ind_hyper)
    x_mero=Dense(50, activation='sigmoid')(ind_mero)
    x_random=Dense(50, activation='sigmoid')(ind_random)
    x_syn=Dense(50, activation='sigmoid')(ind_syn)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    out_syn=Dense(1, activation='sigmoid')(x_syn)
    
    out_coh_hyp=concatenate([out_coh,out_hyper])
    out_coh_mero=concatenate([out_coh,out_mero])
    out_coh_syn=concatenate([out_coh,out_mero])
    out_hyp_mero=concatenate([out_hyper,out_mero])
    out_hyp_syn=concatenate([out_hyper,out_syn])
    out_mero_syn=concatenate([out_mero,out_syn])
    
    out_coh_pre=concatenate([out_coh_hyp,out_coh_mero,out_coh_syn])
    out_hyp_pre=concatenate([out_coh_hyp,out_hyp_mero,out_hyp_syn])
    out_mero_pre=concatenate([out_coh_mero,out_hyp_mero,out_mero_syn])
    out_syn_pre=concatenate([out_coh_syn,out_hyp_syn,out_mero_syn])
    
    out_coh_f=Dense(1, activation='sigmoid')(out_coh_pre)
    out_hyp_f=Dense(1, activation='sigmoid')(out_hyp_pre)
    out_mero_f=Dense(1, activation='sigmoid')(out_mero_pre)
    out_syn_f=Dense(1, activation='sigmoid')(out_syn_pre)
    
    #out_pre=concatenate([x_coh,x_hyper,x_mero,x_random,x_syn])


    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs], outputs=[out_coh,out_hyper,out_mero,out_random,out_syn])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f])
    return model

In [67]:
def NN_Baseline_Fully_shared():
    #l'apprentissage de chaque partie est independant, c'est du vrai multi task
    inputs = Input(shape=(600,))
    
    x_shared=Dense(300, activation='sigmoid')(inputs)
    
    
    
    x_coh=Dense(50, activation='sigmoid')(x_shared)
    x_hyper=Dense(50, activation='sigmoid')(x_shared)
    x_mero=Dense(50, activation='sigmoid')(x_shared)
    x_random=Dense(50, activation='sigmoid')(x_shared)
    x_syn=Dense(50, activation='sigmoid')(x_shared)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    out_syn=Dense(1, activation='sigmoid')(x_syn)
    

    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs], outputs=[out_coh,out_hyper,out_mero,out_random,out_syn])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f])
    return model

In [23]:
custom_activation = lambda x: K.sigmoid(x**2)

In [24]:
def NN_Baseline_Multi_Task_activation():
    #l'apprentissage de chaque partie est independant, c'est du vrai multi task
    inputs = Input(shape=(600,))
    
    x_shared=Dense(300, activation=custom_activation)(inputs)
    
    ind_coh=concatenate([inputs,x_shared])
    ind_hyper=concatenate([inputs,x_shared])
    ind_mero=concatenate([inputs,x_shared])
    ind_random=concatenate([inputs,x_shared])
    ind_syn=concatenate([inputs,x_shared])
    
    x_coh=Dense(50, activation=custom_activation)(ind_coh)
    x_hyper=Dense(50, activation=custom_activation)(ind_hyper)
    x_mero=Dense(50, activation=custom_activation)(ind_mero)
    x_random=Dense(50, activation=custom_activation)(ind_random)
    x_syn=Dense(50, activation=custom_activation)(ind_syn)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    out_syn=Dense(1, activation='sigmoid')(x_syn)
    

    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs], outputs=[out_coh,out_hyper,out_mero,out_random,out_syn])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f])
    return model

In [25]:
def NN_Baseline_Multi_Task_activation2():
    #l'apprentissage de chaque partie est independant, c'est du vrai multi task
    inputs = Input(shape=(600,))
    
    x_shared=Dense(50, activation=custom_activation)(inputs)
    ind=concatenate([inputs,x_shared])
    #IS RANDOm

    x_random=Dense(50, activation=custom_activation)(ind)
    out_random=Dense(1, activation='sigmoid')(x_random)
    
    #IS COH_HYP
    x_COH_HYP=Dense(50, activation=custom_activation)(ind)
    out_COH_HYP=Dense(1, activation='sigmoid')(x_COH_HYP)
    
    #IS COH_MERO
    x_COH_MERO=Dense(50, activation=custom_activation)(ind)
    out_COH_MERO=Dense(1, activation='sigmoid')(x_COH_MERO)
    
    #IS COH_SYN
    x_COH_SYN=Dense(50, activation=custom_activation)(ind)
    out_COH_SYN=Dense(1, activation='sigmoid')(x_COH_SYN)
    
    #IS SYN_MERO
    x_SYN_MERO=Dense(50, activation=custom_activation)(ind)
    out_SYN_MERO=Dense(1, activation='sigmoid')(x_SYN_MERO)
    
    #IS SYN_HYP
    x_SYN_HYP=Dense(50, activation=custom_activation)(ind)
    out_SYN_HYP=Dense(1, activation='sigmoid')(x_SYN_HYP)
    
    #IS MERO_HYP
    x_MERO_HYP=Dense(50, activation=custom_activation)(ind)
    out_MERO_HYP=Dense(1, activation='sigmoid')(x_MERO_HYP)
    
    
    #IS COH
    out_coh_pre=concatenate([out_COH_HYP,out_COH_MERO,out_COH_SYN])
    out_coh=Dense(1, activation='sigmoid')(out_coh_pre)
    #IS HYP
    out_hyp_pre=concatenate([out_COH_HYP,out_MERO_HYP,out_SYN_HYP])
    out_hyper=Dense(1, activation='sigmoid')(out_hyp_pre)
    #IS MERO
    out_mero_pre=concatenate([out_COH_MERO,out_MERO_HYP,out_SYN_MERO])
    out_mero=Dense(1, activation='sigmoid')(out_mero_pre)                        
    #IS SYN
    out_syn_pre=concatenate([out_COH_SYN,out_SYN_HYP,out_SYN_MERO])
    out_syn=Dense(1, activation='sigmoid')(out_syn_pre)
    
    

    
    
    
    

    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs], outputs=[out_coh,out_hyper,out_mero,out_random,out_syn])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f])
    return model

In [26]:
def NN_Baseline_Multi_Task_activation_bis():
    #l'apprentissage de chaque partie est independant, c'est du vrai multi task
    inputs = Input(shape=(600,))
    
   
    

    
    x_coh=Dense(50, activation=custom_activation)(inputs)
    x_hyper=Dense(50, activation=custom_activation)(inputs)
    x_mero=Dense(50, activation=custom_activation)(inputs)
    x_random=Dense(50, activation=custom_activation)(inputs)
    x_syn=Dense(50, activation=custom_activation)(inputs)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    out_syn=Dense(1, activation='sigmoid')(x_syn)
    

    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs], outputs=[out_coh,out_hyper,out_mero,out_random,out_syn])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f])
    return model

In [27]:
def NN_Baseline_Multi_Task_2a2():
    inputs = Input(shape=(600,))
    
    x_shared_syn_hyper=Dense(300, activation='sigmoid')(inputs)
    x_shared_syn_mero=Dense(300, activation='sigmoid')(inputs)
    x_shared_syn_random=Dense(300, activation='sigmoid')(inputs)
    x_shared_syn_cohy=Dense(300, activation='sigmoid')(inputs)
    
    
    ind_coh=concatenate([inputs,x_shared_syn_cohy])
    ind_hyper=concatenate([inputs,x_shared_syn_hyper])
    ind_mero=concatenate([inputs,x_shared_syn_mero])
    ind_random=concatenate([inputs,x_shared_syn_random])
    
    ind_syn1=concatenate([inputs,x_shared_syn_cohy])
    ind_syn2=concatenate([inputs,x_shared_syn_hyper])
    ind_syn3=concatenate([inputs,x_shared_syn_mero])
    ind_syn4=concatenate([inputs,x_shared_syn_random])
    
    x_coh=Dense(50, activation='sigmoid')(ind_coh)
    x_hyper=Dense(50, activation='sigmoid')(ind_hyper)
    x_mero=Dense(50, activation='sigmoid')(ind_mero)
    x_random=Dense(50, activation='sigmoid')(ind_random)
    
    
    x_syn1=Dense(50, activation='sigmoid')(ind_syn1)
    x_syn2=Dense(50, activation='sigmoid')(ind_syn2)
    x_syn3=Dense(50, activation='sigmoid')(ind_syn3)
    x_syn4=Dense(50, activation='sigmoid')(ind_syn4)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    
    out_syn1=Dense(1, activation='sigmoid')(x_syn1)
    out_syn2=Dense(1, activation='sigmoid')(x_syn2)
    out_syn3=Dense(1, activation='sigmoid')(x_syn3)
    out_syn4=Dense(1, activation='sigmoid')(x_syn4)
    
    out_syn_pre=concatenate([out_syn1,out_syn2,out_syn3,out_syn4])
    out_syn=Dense(1, activation='sigmoid')(out_syn_pre)
    
    out_pre=concatenate([out_coh,out_hyper,out_mero,out_random,out_syn])
    #out_final=Dense(5,activation='softmax')(out_pre)
    out_final=out_pre
    model = Model(inputs=[inputs], outputs=[out_final])
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    return model

In [28]:
def NN_Baseline_Multi_Task_2a2_task():
    inputs = Input(shape=(600,))
    
    x_shared_syn_hyper=Dense(300, activation='sigmoid')(inputs)
    x_shared_syn_mero=Dense(300, activation='sigmoid')(inputs)
    x_shared_syn_random=Dense(300, activation='sigmoid')(inputs)
    x_shared_syn_cohy=Dense(300, activation='sigmoid')(inputs)
    
    
    ind_coh=concatenate([inputs,x_shared_syn_cohy])
    ind_hyper=concatenate([inputs,x_shared_syn_hyper])
    ind_mero=concatenate([inputs,x_shared_syn_mero])
    ind_random=concatenate([inputs,x_shared_syn_random])
    
    ind_syn1=concatenate([inputs,x_shared_syn_cohy])
    ind_syn2=concatenate([inputs,x_shared_syn_hyper])
    ind_syn3=concatenate([inputs,x_shared_syn_mero])
    ind_syn4=concatenate([inputs,x_shared_syn_random])
    
    x_coh=Dense(50, activation='sigmoid')(ind_coh)
    x_hyper=Dense(50, activation='sigmoid')(ind_hyper)
    x_mero=Dense(50, activation='sigmoid')(ind_mero)
    x_random=Dense(50, activation='sigmoid')(ind_random)
    
    
    x_syn1=Dense(50, activation='sigmoid')(ind_syn1)
    x_syn2=Dense(50, activation='sigmoid')(ind_syn2)
    x_syn3=Dense(50, activation='sigmoid')(ind_syn3)
    x_syn4=Dense(50, activation='sigmoid')(ind_syn4)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    
    out_syn1=Dense(1, activation='sigmoid')(x_syn1)
    out_syn2=Dense(1, activation='sigmoid')(x_syn2)
    out_syn3=Dense(1, activation='sigmoid')(x_syn3)
    out_syn4=Dense(1, activation='sigmoid')(x_syn4)
    
    out_syn_pre=concatenate([out_syn1,out_syn2,out_syn3,out_syn4])
    out_syn=Dense(1, activation='sigmoid')(out_syn_pre)
    

    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs], outputs=[out_coh,out_hyper,out_mero,out_random,out_syn])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f])
    return model

In [36]:
list_cat=['COH','HYPER','MERO','RANDOM','SYN']

In [23]:
def train_model(name_model,epochs):
    model=name_model
    model.fit(xtrain, ytrain,
                epochs=epochs, validation_data=(xvalid, yvalid),  verbose=True, callbacks=[EarlyStopping(patience=5)])
    return model

In [24]:
def train_model_task(name_model,epochs):
    model=name_model
    model.fit(xtrain, [ytrain.transpose()[0],ytrain.transpose()[1],ytrain.transpose()[2]
                       ,ytrain.transpose()[3],ytrain.transpose()[4]],
                epochs=epochs, validation_data=(xvalid, [yvalid.transpose()[0],yvalid.transpose()[1],yvalid.transpose()[2]
                       ,yvalid.transpose()[3],yvalid.transpose()[4]]),  verbose=False, callbacks=[EarlyStopping(patience=5)])
    return model

In [25]:
def train_model_task_new_idea(name_model,epochs):
    model=name_model
    model.fit(xtrain, [ytrain.transpose()[0],ytrain.transpose()[1],ytrain.transpose()[2]
                       ,ytrain.transpose()[3],ytrain.transpose()[4],ytrain],
                epochs=epochs, validation_data=(xvalid, [yvalid.transpose()[0],yvalid.transpose()[1],yvalid.transpose()[2]
                       ,yvalid.transpose()[3],yvalid.transpose()[4],yvalid]),  verbose=False, callbacks=[EarlyStopping(patience=5)])
    return model

In [26]:
#accuracy_score(ytest.transpose()[1], (preds[1]> 0.5).astype(int))

In [27]:
def func_pred_task(model,xtest):
    preds = model.predict(xtest, verbose=False)
    preds_pre=[]
    for i in range(len(preds[0])):
        a=np.zeros(5)
        b=[]
        for j in range(5):
            b.append(preds[j][i])
        a[np.argmax(b)]=1
        preds_pre.append(a)
    preds_all=[list_cat[np.argmax(l)] for l in preds_pre]
    return preds_all
    

In [28]:
def test_maxf1(model,xtest):
    preds = model.predict(xtest, verbose=False)
    seuils=np.arange(0,1,0.05)
    for i in range(len(preds)):
        print('*********')
        print(list_cat[i])
        print('*********')
        f1_final=[]
        for seuil in seuils :
            f1_score(ytest.transpose()[i], (preds[i]> seuil).astype(int))
            f1_final.append(f1_score(ytest.transpose()[i], (preds[i]> seuil).astype(int)))
        print(max(f1_final))
        c=np.argmax(f1_final)
        print(seuils[c])
            
    

In [29]:
def func_preds(model,xtest):
    preds = model.predict(xtest, verbose=False)
    preds_all=[list_cat[np.argmax(l)] for l in preds]
    return preds_all

In [30]:
def func_preds_new_idea(model,xtest):
    preds = model.predict(xtest, verbose=False)
    preds=preds[-1]
    preds_all=[list_cat[np.argmax(l)] for l in preds]
    return preds_all

In [59]:
def func_preds_new_idea2(model,xtest):
    preds = model.predict(xtest, verbose=False)
    preds=preds[0:5]
    preds_pre=[]
    for i in range(len(preds[0])):
        a=np.zeros(5)
        b=[]
        for j in range(5):
            b.append(preds[j][i])
        a[np.argmax(b)]=1
        preds_pre.append(a)
    preds_all=[list_cat[np.argmax(l)] for l in preds_pre]
    return preds_all

In [31]:
def create_metrics(y_test,y_pred):
    y_test=[list_cat[np.argmax(l)] for l in y_test]
    cm = ConfusionMatrix(actual_vector=y_test, predict_vector=y_pred) # Create CM From Data
    dict_F1=cm.class_stat['F1']
    df_metrics=pd.DataFrame.from_dict(dict_F1, orient='index',columns=['F1'])
    df_metrics['ACC']=pd.DataFrame.from_dict(cm.class_stat['ACC'], orient='index' )

    #cm.class_stat
    return df_metrics

In [32]:
def main(name_model,xtest,epochs):
    model=train_model(name_model,epochs)
    preds_all=func_preds(model,xtest)
    df_metrics=create_metrics(ytest,preds_all)
    return df_metrics
    

In [33]:
def main_task(name_model,xtest,epochs):
    model=train_model_task(name_model,epochs)
    preds_all=func_pred_task(model,xtest)
    test_maxf1(model,xtest)
    df_metrics=create_metrics(ytest,preds_all)
    return df_metrics

In [34]:
def main_task_new_idea(name_model,xtest,epochs):
    model=train_model_task_new_idea(name_model,epochs)
    preds_all=func_preds_new_idea(model,xtest)
    df_metrics=create_metrics(ytest,preds_all)
    return df_metrics

In [60]:
def main_task_new_idea2(name_model,xtest,epochs):
    model=train_model_task_new_idea(name_model,epochs)
    preds_all=func_preds_new_idea2(model,xtest)
    df_metrics=create_metrics(ytest,preds_all)
    return df_metrics

In [63]:
df_adv=main_task_new_idea2(NN_Baseline_Multi_Task_indep_adv(),xtest,50)

Predict      COH          HYPER        MERO         RANDOM       SYN          
Actual
COH          293          146          23           127          89           

HYPER        106          932          40           483          240          

MERO         33           71           220          100          30           

RANDOM       177          386          72           3067         216          

SYN          102          294          10           301          310          





Overall Statistics : 

95% CI                                                            (0.6021,0.62363)
ACC Macro                                                         0.84514
AUNP                                                              0.71729
AUNU                                                              0.69564
Bennett S                                                         0.51608
CBA                                                               0.49263
CSI                               

In [64]:
df_adv

,F1,ACC
COH,0.421886,0.897941
HYPER,0.513499,0.775547
MERO,0.537241,0.951830
RANDOM,0.767134,0.763345
SYN,0.325973,0.837062


In [136]:
df_new_idea=main_task_new_idea(NN_Baseline_Multi_Task_indep_new_idea(),xtest,50)

In [137]:
df_new_idea

,F1,ACC
COH,0.436662,0.903915
HYPER,0.503047,0.771988
MERO,0.488404,0.952339
RANDOM,0.760422,0.751652
SYN,0.310290,0.833884


In [39]:
df_NN_Multi=main(NN_Baseline_Multi(),xtest,50)


Train on 13517 samples, validate on 5793 samples
Epoch 1/50
13517/13517 [==============================] - 1s 91us/step - loss: 1.1333 - val_loss: 1.0196
Epoch 2/50
13517/13517 [==============================] - 1s 54us/step - loss: 0.9690 - val_loss: 0.9741
Epoch 3/50
13517/13517 [==============================] - 1s 96us/step - loss: 0.9220 - val_loss: 0.9656
Epoch 4/50
13517/13517 [==============================] - 1s 54us/step - loss: 0.8933 - val_loss: 0.9487
Epoch 5/50
13517/13517 [==============================] - 1s 55us/step - loss: 0.8706 - val_loss: 0.9350
Epoch 6/50
13517/13517 [==============================] - 1s 83us/step - loss: 0.8477 - val_loss: 0.9277
Epoch 7/50
13517/13517 [==============================] - 1s 88us/step - loss: 0.8249 - val_loss: 0.9114
Epoch 8/50
13517/13517 [==============================] - 1s 79us/step - loss: 0.7989 - val_loss: 0.8949
Epoch 9/50
13517/13517 [==============================] - 1s 93us/step - loss: 0.7725 - val_loss: 0.8780
Epoch

In [40]:
df_NN_Multi

,F1,ACC
COH,0.414045,0.898195
HYPER,0.474820,0.768048
MERO,0.510204,0.951195
RANDOM,0.726097,0.720005
SYN,0.304790,0.817361


In [48]:
df_NN_Multi_Task=main(NN_Baseline_Multi_Task(),xtest,50)

Train on 13517 samples, validate on 5793 samples
Epoch 1/50
13517/13517 [==============================] - 3s 255us/step - loss: 1.0477 - val_loss: 0.9685
Epoch 2/50
13517/13517 [==============================] - 3s 213us/step - loss: 0.9048 - val_loss: 0.9200
Epoch 3/50
13517/13517 [==============================] - 3s 213us/step - loss: 0.8232 - val_loss: 0.8690
Epoch 4/50
13517/13517 [==============================] - 3s 233us/step - loss: 0.7406 - val_loss: 0.7971
Epoch 5/50
13517/13517 [==============================] - 3s 215us/step - loss: 0.6653 - val_loss: 0.7626
Epoch 6/50
13517/13517 [==============================] - 3s 225us/step - loss: 0.5931 - val_loss: 0.7274
Epoch 7/50
13517/13517 [==============================] - 3s 219us/step - loss: 0.5287 - val_loss: 0.7178
Epoch 8/50
13517/13517 [==============================] - 3s 217us/step - loss: 0.4710 - val_loss: 0.6873
Epoch 9/50
13517/13517 [==============================] - 3s 213us/step - loss: 0.4185 - val_loss: 0.67

In [49]:
df_NN_Multi_Task

,F1,ACC
COH,0.432470,0.896924
HYPER,0.486615,0.780630
MERO,0.517533,0.949288
RANDOM,0.746065,0.749873
SYN,0.336993,0.800458


In [43]:
df_NN_Multi_Task_2a2=main(NN_Baseline_Multi_Task_2a2(),xtest,50)

Train on 13517 samples, validate on 5793 samples
Epoch 1/50
13517/13517 [==============================] - 7s 491us/step - loss: 1.0632 - val_loss: 0.9955
Epoch 2/50
13517/13517 [==============================] - 6s 446us/step - loss: 0.9129 - val_loss: 0.9197
Epoch 3/50
13517/13517 [==============================] - 6s 460us/step - loss: 0.8357 - val_loss: 0.8675
Epoch 4/50
13517/13517 [==============================] - 6s 449us/step - loss: 0.7498 - val_loss: 0.8419
Epoch 5/50
13517/13517 [==============================] - 6s 447us/step - loss: 0.6742 - val_loss: 0.7864
Epoch 6/50
13517/13517 [==============================] - 6s 450us/step - loss: 0.6074 - val_loss: 0.7839
Epoch 7/50
13517/13517 [==============================] - 6s 452us/step - loss: 0.5446 - val_loss: 0.7530
Epoch 8/50
13517/13517 [==============================] - 6s 473us/step - loss: 0.4856 - val_loss: 0.7480
Epoch 9/50
13517/13517 [==============================] - 6s 447us/step - loss: 0.4268 - val_loss: 0.73

In [45]:
df_NN_Multi_Task_2a2

,F1,ACC
COH,0.412037,0.903152
HYPER,0.474264,0.775419
MERO,0.509895,0.946492
RANDOM,0.740926,0.736019
SYN,0.318656,0.809227


In [69]:
df_Fully=main_task(NN_Baseline_Fully_shared(),xtest,50)

*********
COH
*********
0.41560390097524386
0.30000000000000004
*********
HYPER
*********
0.49900438072481085
0.1
*********
MERO
*********
0.5339366515837105
0.05
*********
RANDOM
*********
0.7678244972577697
0.15000000000000002
*********
SYN
*********
0.36472065285624605
0.15000000000000002


In [70]:
df_Fully

,F1,ACC
COH,0.421836,0.911159
HYPER,0.470622,0.768683
MERO,0.524675,0.953482
RANDOM,0.756637,0.748348
SYN,0.331654,0.814565


In [139]:
df_Multi_Task_indep2a2=main_task(NN_Baseline_Multi_Task_indep_new_idea2a2(),xtest,50)

*********
COH
*********
0.4379194630872483
0.45
*********
HYPER
*********
0.5257238075400958
0.1
*********
MERO
*********
0.530521642619312
0.25
*********
RANDOM
*********
0.7679905157083581
0.25
*********
SYN
*********
0.3705293276108727
0.15000000000000002


In [140]:
df_Multi_Task_indep2a2

,F1,ACC
COH,0.434231,0.909253
HYPER,0.480907,0.778851
MERO,0.540476,0.950941
RANDOM,0.766215,0.754448
SYN,0.323092,0.826385


In [39]:
df_oneVsRest=main_task(NN_Baseline_oneVsRest(),xtest,50)

*********
COH
*********
0.43454038997214484
0.30000000000000004
*********
HYPER
*********
0.5093917710196779
0.2
*********
MERO
*********
0.5270403146509341
0.05
*********
RANDOM
*********
0.7574418604651163
0.2
*********
SYN
*********
0.34730538922155685
0.15000000000000002


In [41]:
df_oneVsRest

,F1,ACC
COH,0.457014,0.908490
HYPER,0.514884,0.776309
MERO,0.541353,0.953482
RANDOM,0.752537,0.748983
SYN,0.326510,0.828546


In [56]:
df_Multi_Task_indep=main_task(NN_Baseline_Multi_Task_indep(),xtest,50)

*********
COH
*********
0.42652329749103945
0.8
*********
HYPER
*********
0.5118191161356629
0.1
*********
MERO
*********
0.5227537922987165
0.2
*********
RANDOM
*********
0.7558891454965357
0.30000000000000004
*********
SYN
*********
0.35991957104557637
0.1
Predict      COH          HYPER        MERO         RANDOM       SYN          
Actual
COH          305          97           15           171          90           

HYPER        101          833          40           599          228          

MERO         39           62           205          120          28           

RANDOM       189          358          64           3081         226          

SYN          95           258          10           367          287          





Overall Statistics : 

95% CI                                                            (0.58792,0.60959)
ACC Macro                                                         0.8395
AUNP                                                              0.697

In [57]:
df_Multi_Task_indep

,F1,ACC
COH,0.433547,0.898704
HYPER,0.488706,0.778470
MERO,0.520305,0.951957
RANDOM,0.746366,0.733859
SYN,0.305970,0.834520


In [54]:
#shared1 mero syn coh hyp, shared2 hyp rand
df_Multi_Task_indep

,F1,ACC
COH,0.444276,0.906838
HYPER,0.493932,0.772115
MERO,0.560000,0.956660
RANDOM,0.758930,0.753813
SYN,0.321705,0.822064


In [48]:
#w/o random in shared
df_Multi_Task_indep

,F1,ACC
COH,0.449012,0.911413
HYPER,0.466645,0.789654
MERO,0.534527,0.953737
RANDOM,0.752285,0.731317
SYN,0.344055,0.828927


In [131]:
#shared all
df_Multi_Task_indep

,F1,ACC
COH,0.414091,0.896416
HYPER,0.500000,0.783172
MERO,0.517197,0.951830
RANDOM,0.767049,0.759914
SYN,0.312898,0.821937


In [44]:
df_Multi_Task_2a2_indep=main_task(NN_Baseline_Multi_Task_2a2_task(),xtest,50)

In [45]:
df_Multi_Task_Activation2=main_task(NN_Baseline_Multi_Task_activation2(),xtest,50)

*********
COH
*********
0.4197111299915038
0.35000000000000003
*********
HYPER
*********
0.4956634865568084
0.05
*********
MERO
*********


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.5161290322580645
0.15000000000000002
*********
RANDOM
*********
0.757000474608448
0.05
*********
SYN
*********
0.3496932515337423
0.15000000000000002
['COH', 'HYPER', 'MERO', 'RANDOM', 'SYN']


In [46]:
df_Multi_Task_Activation2

,F1,ACC
COH,0.442260,0.913447
HYPER,0.473157,0.779232
MERO,0.522546,0.954245
RANDOM,0.745443,0.746187
SYN,0.332952,0.777961


In [47]:
df_NN_Multi

,F1,ACC
COH,0.414045,0.898195
HYPER,0.474820,0.768048
MERO,0.510204,0.951195
RANDOM,0.726097,0.720005
SYN,0.304790,0.817361


In [48]:
df_Multi_Task_indep

,F1,ACC
COH,0.437554,0.917006
HYPER,0.498299,0.775038
MERO,0.546108,0.951830
RANDOM,0.760195,0.755592
SYN,0.343821,0.817107


In [49]:
df_Multi_Task_Activation=main_task(NN_Baseline_Multi_Task_activation_bis(),xtest,50)

*********
COH
*********
0.4364548494983277
0.2
*********
HYPER
*********
0.4916067146282974
0.1
*********
MERO
*********
0.5064782096584217
0.05
*********
RANDOM
*********
0.7608388312912348
0.4
*********
SYN
*********
0.3546991742036964
0.15000000000000002
['COH', 'HYPER', 'MERO', 'RANDOM', 'SYN']


In [50]:
df_Multi_Task_Activation

,F1,ACC
COH,0.442308,0.918912
HYPER,0.427721,0.792196
MERO,0.379254,0.951322
RANDOM,0.759159,0.717590
SYN,0.332981,0.839603


In [51]:
df_Multi_Task_Activation

,F1,ACC
COH,0.442308,0.918912
HYPER,0.427721,0.792196
MERO,0.379254,0.951322
RANDOM,0.759159,0.717590
SYN,0.332981,0.839603


In [52]:
df_Multi_Task_indep

,F1,ACC
COH,0.437554,0.917006
HYPER,0.498299,0.775038
MERO,0.546108,0.951830
RANDOM,0.760195,0.755592
SYN,0.343821,0.817107


In [53]:
df_Multi_Task_Activation


,F1,ACC
COH,0.442308,0.918912
HYPER,0.427721,0.792196
MERO,0.379254,0.951322
RANDOM,0.759159,0.717590
SYN,0.332981,0.839603


In [54]:
ytest[0]

array([0, 1, 0, 0, 0], dtype=int64)

In [51]:
df_train=df_all.loc[df_all.is_train==True]

In [52]:
df_train.head()

,w1,w2,CAT_COH,CAT_HYPER,CAT_MERO,CAT_RANDOM,CAT_SYN,known_words,is_train,is_test
1,destructiveness,centaur,0,0,0,1,0,True,True,False
7,knot,burl,0,0,0,0,1,True,True,False
8,pecker,tool,0,0,0,0,1,True,True,False
15,minister,residence,0,0,0,1,0,True,True,False
16,neuritis,holly,0,0,0,1,0,True,True,False


In [53]:
df_RANDOM=df_train[df_train.CAT_RANDOM==1]
df_COH=df_train[df_train.CAT_COH==1]
df_HYPER=df_train[df_train.CAT_HYPER==1]
df_MERO=df_train[df_train.CAT_MERO==1]
df_SYN=df_train[df_train.CAT_SYN==1]

In [54]:
df_SYN.head()

,w1,w2,CAT_COH,CAT_HYPER,CAT_MERO,CAT_RANDOM,CAT_SYN,known_words,is_train,is_test
7,knot,burl,0,0,0,0,1,True,True,False
8,pecker,tool,0,0,0,0,1,True,True,False
81,hardness,insensibility,0,0,0,0,1,True,True,False
135,position,emplacement,0,0,0,0,1,True,True,False
165,vomit,regurgitation,0,0,0,0,1,True,True,False


In [55]:
def append_random(df_COH,name_col):
    df_COH=df_COH.append(df_RANDOM).sample(frac=1).reset_index(drop=True)
    print(df_SYN.head())
    df_COH=df_COH.rename(columns={'CAT_'+name_col:'Category'})
    return df_COH[['w1','w2','Category']]

In [56]:
df_COH=append_random(df_COH,'COH')
df_HYPER=append_random(df_HYPER,'HYPER')
df_MERO=append_random(df_MERO,'MERO')
df_SYN=append_random(df_SYN,'SYN')

           w1             w2  CAT_COH  CAT_HYPER  CAT_MERO  CAT_RANDOM  \
7        knot           burl        0          0         0           0   
8      pecker           tool        0          0         0           0   
81   hardness  insensibility        0          0         0           0   
135  position    emplacement        0          0         0           0   
165     vomit  regurgitation        0          0         0           0   

     CAT_SYN  known_words  is_train  is_test  
7          1         True      True    False  
8          1         True      True    False  
81         1         True      True    False  
135        1         True      True    False  
165        1         True      True    False  
           w1             w2  CAT_COH  CAT_HYPER  CAT_MERO  CAT_RANDOM  \
7        knot           burl        0          0         0           0   
8      pecker           tool        0          0         0           0   
81   hardness  insensibility        0          0   

In [57]:
df_SYN.head()

,w1,w2,Category
0,goat,thermosphere,0
1,dropsy,diltiazem,0
2,nerd,detoxification,0
3,poultry,straddle,0
4,actor,sight,0


In [58]:
def append_to_random(df_RANDOM):
    
    df_RANDOM=df_RANDOM.rename(columns={'CAT_'+'RANDOM':'Category'})
    df_RANDOM=df_RANDOM[['w1','w2','Category']]
    df_RANDOM=df_RANDOM.append(df_COH)
    df_RANDOM=df_RANDOM.append(df_HYPER)
    df_RANDOM=df_RANDOM.append(df_MERO)
    df_RANDOM=df_RANDOM.append(df_SYN)
    df_RANDOM=df_RANDOM.sample(frac=1).reset_index(drop=True)
    
    
    return df_RANDOM

In [59]:
df_RANDOM=append_to_random(df_RANDOM)

In [60]:
xtrain_COH, ytrain_COH = get_vector_representation_of_word_pairs_multi(df_COH, embeddings)
xtrain_HYPER, ytrain_HYPER = get_vector_representation_of_word_pairs_multi(df_HYPER, embeddings)
xtrain_MERO, ytrain_MERO = get_vector_representation_of_word_pairs_multi(df_MERO, embeddings)
xtrain_RANDOM, ytrain_RANDOM = get_vector_representation_of_word_pairs_multi(df_MERO, embeddings)
xtrain_SYN, ytrain_SYN = get_vector_representation_of_word_pairs_multi(df_SYN, embeddings)

In [61]:
def split_valid(xtrain_COH,ytrain_COH):
    
    xtrain_COH, xvalid_COH, ytrain_COH, yvalid_COH = train_test_split(xtrain_COH, ytrain_COH, stratify=ytrain_COH,  test_size=0.30, random_state=1234,)
    return xtrain_COH, xvalid_COH, ytrain_COH, yvalid_COH

In [62]:
xtrain_COH, xvalid_COH, ytrain_COH, yvalid_COH=split_valid(xtrain_COH,ytrain_COH)
xtrain_HYPER, xvalid_HYPER, ytrain_HYPER, yvalid_HYPER=split_valid(xtrain_HYPER,ytrain_HYPER)
xtrain_MERO, xvalid_MERO, ytrain_MERO, yvalid_MERO=split_valid(xtrain_MERO,ytrain_MERO)
xtrain_RANDOM, xvalid_RANDOM, ytrain_RANDOM, yvalid_RANDOM=split_valid(xtrain_RANDOM,ytrain_RANDOM)
xtrain_SYN, xvalid_SYN, ytrain_SYN, yvalid_SYN=split_valid(xtrain_SYN,ytrain_SYN)

In [63]:
def Multi_Task():
    #l'apprentissage de chaque partie est independant, c'est du vrai multi task
    inputs_COH = Input(shape=(600,))
    inputs_HYPER = Input(shape=(600,))
    inputs_MERO = Input(shape=(600,))
    inputs_RANDOM = Input(shape=(600,))
    inputs_SYN = Input(shape=(600,))
    
    x_shared=Dense(300, activation='sigmoid')
    
    ind_coh=concatenate([inputs_COH,x_shared(inputs_COH)])
    ind_hyper=concatenate([inputs_HYPER,x_shared(inputs_HYPER)])
    ind_mero=concatenate([inputs_MERO,x_shared(inputs_MERO)])
    #ind_random=concatenate([inputs_RANDOM,x_shared(inputs_RANDOM)])
    ind_syn=concatenate([inputs_SYN,x_shared(inputs_SYN)])
    
    x_coh=Dense(50, activation='sigmoid')(ind_coh)
    x_hyper=Dense(50, activation='sigmoid')(ind_hyper)
    x_mero=Dense(50, activation='sigmoid')(ind_mero)
    x_random=Dense(50, activation='sigmoid')(inputs_RANDOM)
    x_syn=Dense(50, activation='sigmoid')(ind_syn)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    out_syn=Dense(1, activation='sigmoid')(x_syn)
    
    #y_final=concatenate([out_coh,out_hyper,out_mero,out_random,out_syn])
    #y_final_out=Dense(1, activation='softmax')(y_final)

    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs_COH,inputs_HYPER,inputs_MERO,inputs_RANDOM,inputs_SYN], 
                  outputs=[out_coh,out_hyper,out_mero,out_random,out_syn])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f])
    return model

In [64]:
def Baseline_indep():
    #l'apprentissage de chaque partie est independant, c'est du vrai multi task
    inputs_COH = Input(shape=(600,))
    inputs_HYPER = Input(shape=(600,))
    inputs_MERO = Input(shape=(600,))
    inputs_RANDOM = Input(shape=(600,))
    inputs_SYN = Input(shape=(600,))
    
    

    x_coh=Dense(50, activation='sigmoid')(inputs_COH)
    x_hyper=Dense(50, activation='sigmoid')(inputs_HYPER)
    x_mero=Dense(50, activation='sigmoid')(inputs_MERO)
    x_random=Dense(50, activation='sigmoid')(inputs_RANDOM)
    x_syn=Dense(50, activation='sigmoid')(inputs_SYN)
    
    out_coh=Dense(1, activation='sigmoid')(x_coh)
    out_hyper=Dense(1, activation='sigmoid')(x_hyper)
    out_mero=Dense(1, activation='sigmoid')(x_mero)
    out_random=Dense(1, activation='sigmoid')(x_random)
    out_syn=Dense(1, activation='sigmoid')(x_syn)
    
    #y_final=concatenate([out_coh,out_hyper,out_mero,out_random,out_syn])
    #y_final_out=Dense(1, activation='softmax')(y_final)

    loss_f='binary_crossentropy'
    model = Model(inputs=[inputs_COH,inputs_HYPER,inputs_MERO,inputs_RANDOM,inputs_SYN], 
                  outputs=[out_coh,out_hyper,out_mero,out_random,out_syn])
    model.compile(optimizer='rmsprop', loss= [loss_f,loss_f,loss_f,loss_f,loss_f])
    return model

In [65]:
def train_model_multi_task(name_model,epochs):
    batch=64
    a1=len(xtrain)
    
    idx1 = np.random.choice(np.arange(a1), batch, replace=False)
    
    model=name_model
    val=0
    a=[]
    indice=0

    for it in range(epochs):
        model.fit([xtrain[idx1],xtrain[idx1],xtrain[idx1],xtrain[idx1],xtrain[idx1]] ,
                  [ytrain.transpose()[0][idx1],ytrain.transpose()[1][idx1],ytrain.transpose()[2][idx1]
                       ,ytrain.transpose()[3][idx1],ytrain.transpose()[4][idx1]],
                    epochs=1, 

                  verbose=False)
    
        if it %100 == 0 :
            preds_all= preds_multi_task2(model,xtest)
            df_metrics=create_metrics(ytest,preds_all)
            val_new=df_metrics['F1'].sum()
            if val_new>val:
                a=df_metrics
                indice=it
        
    
    return a,indice

In [66]:
a,indice=train_model_multi_task(Multi_Task(),10000)

NameError: name 'preds_multi_task2' is not defined

In [123]:
a

,F1,ACC
COH,0.092840,0.853457
HYPER,0.167710,0.746441
MERO,0.124031,0.942552
RANDOM,0.624134,0.544738
SYN,0.189911,0.791815


In [94]:
a,indice=train_model_multi_task(Baseline_indep(),5000)

In [122]:
a

,F1,ACC
COH,0.092840,0.853457
HYPER,0.167710,0.746441
MERO,0.124031,0.942552
RANDOM,0.624134,0.544738
SYN,0.189911,0.791815


In [77]:
def train_model_multi_task(name_model,epochs):
    batch=64
    a1=len(xtrain_COH)
    a2=len(xtrain_HYPER)
    a3=len(xtrain_MERO)
    a4=len(xtrain_RANDOM)
    a5=len(xtrain_SYN)
    idx1 = np.random.choice(np.arange(a1), batch, replace=False)
    idx2 = np.random.choice(np.arange(a2), batch, replace=False)
    idx3 = np.random.choice(np.arange(a3), batch, replace=False)
    idx4 = np.random.choice(np.arange(a4), batch, replace=False)
    idx5 = np.random.choice(np.arange(a5), batch, replace=False)
    model=name_model
    for it in range(epochs):
        model.fit([xtrain_COH[idx1],xtrain_HYPER[idx2],xtrain_MERO[idx3],xtrain_RANDOM[idx4],xtrain_SYN[idx5]] ,
                  [ytrain_COH[idx1],ytrain_HYPER[idx2],ytrain_MERO[idx3],ytrain_RANDOM[idx4],ytrain_SYN[idx5]],
                    epochs=1, 

                  verbose=True, 
                  callbacks=[EarlyStopping(patience=5)])
    return model

In [336]:
#validation_data=([xvalid_COH,xvalid_HYPER,xvalid_MERO,xvalid_SYN], 
#                               [yvalid_COH,yvalid_HYPER,yvalid_MERO,yvalid_SYN]),

In [339]:
model_multi_task=train_model_multi_task(Multi_Task(),1000)

Epoch 1/1
64/64 [==============================] - 1s 13ms/step - loss: 3.4177 - dense_361_loss: 0.6395 - dense_362_loss: 0.8434 - dense_363_loss: 0.4240 - dense_364_loss: 1.0175 - dense_365_loss: 0.4933
Epoch 1/1
64/64 [==============================] - 0s 436us/step - loss: 2.2785 - dense_361_loss: 0.3325 - dense_362_loss: 0.5777 - dense_363_loss: 0.1974 - dense_364_loss: 0.7899 - dense_365_loss: 0.3809
Epoch 1/1
64/64 [==============================] - 0s 608us/step - loss: 1.9700 - dense_361_loss: 0.2759 - dense_362_loss: 0.5206 - dense_363_loss: 0.1582 - dense_364_loss: 0.6731 - dense_365_loss: 0.3422
Epoch 1/1
64/64 [==============================] - 0s 436us/step - loss: 1.7678 - dense_361_loss: 0.2420 - dense_362_loss: 0.4711 - dense_363_loss: 0.1346 - dense_364_loss: 0.5872 - dense_365_loss: 0.3329
Epoch 1/1
32/64 [==============>...............] - ETA: 0s - loss: 1.6401 - dense_361_loss: 0.2101 - dense_362_loss: 0.4427 - dense_363_loss: 0.1656 - dense_364_loss: 0.5456 - dense

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,dense_361_loss,dense_362_loss,dense_363_loss,dense_364_loss,dense_365_loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


64/64 [==============================] - 0s 654us/step - loss: 1.5754 - dense_361_loss: 0.2162 - dense_362_loss: 0.4386 - dense_363_loss: 0.1166 - dense_364_loss: 0.5245 - dense_365_loss: 0.2795
Epoch 1/1
64/64 [==============================] - 0s 405us/step - loss: 1.4293 - dense_361_loss: 0.1917 - dense_362_loss: 0.4091 - dense_363_loss: 0.0995 - dense_364_loss: 0.4744 - dense_365_loss: 0.2545
Epoch 1/1
64/64 [==============================] - 0s 701us/step - loss: 1.2941 - dense_361_loss: 0.1692 - dense_362_loss: 0.3712 - dense_363_loss: 0.0868 - dense_364_loss: 0.4310 - dense_365_loss: 0.2359
Epoch 1/1
64/64 [==============================] - 0s 436us/step - loss: 1.1772 - dense_361_loss: 0.1538 - dense_362_loss: 0.3383 - dense_363_loss: 0.0767 - dense_364_loss: 0.3960 - dense_365_loss: 0.2124
Epoch 1/1
64/64 [==============================] - 0s 639us/step - loss: 1.0717 - dense_361_loss: 0.1366 - dense_362_loss: 0.3130 - dense_363_loss: 0.0672 - dense_364_loss: 0.3630 - dense_36

64/64 [==============================] - 0s 639us/step - loss: 0.0093 - dense_361_loss: 0.0010 - dense_362_loss: 0.0027 - dense_363_loss: 6.3603e-04 - dense_364_loss: 0.0042 - dense_365_loss: 7.9818e-04
Epoch 1/1
64/64 [==============================] - 0s 748us/step - loss: 0.0089 - dense_361_loss: 9.6123e-04 - dense_362_loss: 0.0026 - dense_363_loss: 6.0782e-04 - dense_364_loss: 0.0040 - dense_365_loss: 7.6251e-04
Epoch 1/1
64/64 [==============================] - 0s 670us/step - loss: 0.0085 - dense_361_loss: 9.2159e-04 - dense_362_loss: 0.0025 - dense_363_loss: 5.8337e-04 - dense_364_loss: 0.0038 - dense_365_loss: 7.2942e-04
Epoch 1/1
64/64 [==============================] - 0s 826us/step - loss: 0.0081 - dense_361_loss: 8.8263e-04 - dense_362_loss: 0.0024 - dense_363_loss: 5.5959e-04 - dense_364_loss: 0.0036 - dense_365_loss: 6.9874e-04
Epoch 1/1
64/64 [==============================] - 0s 701us/step - loss: 0.0078 - dense_361_loss: 8.4593e-04 - dense_362_loss: 0.0024 - dense_363_

64/64 [==============================] - 0s 717us/step - loss: 5.1186e-04 - dense_361_loss: 5.3032e-05 - dense_362_loss: 2.3086e-04 - dense_363_loss: 3.9807e-05 - dense_364_loss: 1.3525e-04 - dense_365_loss: 5.2910e-05
Epoch 1/1
64/64 [==============================] - 0s 764us/step - loss: 4.9346e-04 - dense_361_loss: 5.1010e-05 - dense_362_loss: 2.2328e-04 - dense_363_loss: 3.8175e-05 - dense_364_loss: 1.2981e-04 - dense_365_loss: 5.1188e-05
Epoch 1/1
64/64 [==============================] - 0s 826us/step - loss: 4.7629e-04 - dense_361_loss: 4.9050e-05 - dense_362_loss: 2.1596e-04 - dense_363_loss: 3.6798e-05 - dense_364_loss: 1.2505e-04 - dense_365_loss: 4.9432e-05
Epoch 1/1
64/64 [==============================] - 0s 670us/step - loss: 4.5912e-04 - dense_361_loss: 4.7150e-05 - dense_362_loss: 2.0888e-04 - dense_363_loss: 3.5476e-05 - dense_364_loss: 1.1982e-04 - dense_365_loss: 4.7798e-05
Epoch 1/1
64/64 [==============================] - 0s 514us/step - loss: 4.4260e-04 - dense_36

...

64/64 [==============================] - 0s 919us/step - loss: 6.4719e-08 - dense_361_loss: 1.1315e-08 - dense_362_loss: 1.5779e-08 - dense_363_loss: 1.1650e-08 - dense_364_loss: 1.2965e-08 - dense_365_loss: 1.3011e-08
Epoch 1/1
64/64 [==============================] - 0s 764us/step - loss: 6.4574e-08 - dense_361_loss: 1.1282e-08 - dense_362_loss: 1.5769e-08 - dense_363_loss: 1.1622e-08 - dense_364_loss: 1.2929e-08 - dense_365_loss: 1.2972e-08
Epoch 1/1
64/64 [==============================] - 0s 810us/step - loss: 6.4427e-08 - dense_361_loss: 1.1250e-08 - dense_362_loss: 1.5760e-08 - dense_363_loss: 1.1594e-08 - dense_364_loss: 1.2893e-08 - dense_365_loss: 1.2930e-08
Epoch 1/1
64/64 [==============================] - 0s 795us/step - loss: 6.4286e-08 - dense_361_loss: 1.1217e-08 - dense_362_loss: 1.5748e-08 - dense_363_loss: 1.1570e-08 - dense_364_loss: 1.2857e-08 - dense_365_loss: 1.2894e-08
Epoch 1/1
64/64 [==============================] - 0s 732us/step - loss: 6.4138e-08 - dense_36

64/64 [==============================] - 0s 857us/step - loss: 5.5701e-08 - dense_361_loss: 9.3112e-09 - dense_362_loss: 1.5145e-08 - dense_363_loss: 9.9020e-09 - dense_364_loss: 1.0755e-08 - dense_365_loss: 1.0588e-08
Epoch 1/1
64/64 [==============================] - 0s 951us/step - loss: 5.5590e-08 - dense_361_loss: 9.2899e-09 - dense_362_loss: 1.5138e-08 - dense_363_loss: 9.8766e-09 - dense_364_loss: 1.0729e-08 - dense_365_loss: 1.0557e-08
Epoch 1/1
64/64 [==============================] - 0s 764us/step - loss: 5.5479e-08 - dense_361_loss: 9.2621e-09 - dense_362_loss: 1.5131e-08 - dense_363_loss: 9.8543e-09 - dense_364_loss: 1.0703e-08 - dense_365_loss: 1.0529e-08
Epoch 1/1
64/64 [==============================] - 0s 717us/step - loss: 5.5362e-08 - dense_361_loss: 9.2386e-09 - dense_362_loss: 1.5120e-08 - dense_363_loss: 9.8322e-09 - dense_364_loss: 1.0674e-08 - dense_365_loss: 1.0497e-08
Epoch 1/1
64/64 [==============================] - 0s 857us/step - loss: 5.5249e-08 - dense_36

64/64 [==============================] - 0s 764us/step - loss: 4.8682e-08 - dense_361_loss: 7.7713e-09 - dense_362_loss: 1.4675e-08 - dense_363_loss: 8.4875e-09 - dense_364_loss: 9.0564e-09 - dense_365_loss: 8.6916e-09
Epoch 1/1
64/64 [==============================] - 0s 732us/step - loss: 4.8592e-08 - dense_361_loss: 7.7509e-09 - dense_362_loss: 1.4670e-08 - dense_363_loss: 8.4693e-09 - dense_364_loss: 9.0347e-09 - dense_365_loss: 8.6678e-09
Epoch 1/1
64/64 [==============================] - 0s 732us/step - loss: 4.8505e-08 - dense_361_loss: 7.7326e-09 - dense_362_loss: 1.4665e-08 - dense_363_loss: 8.4511e-09 - dense_364_loss: 9.0130e-09 - dense_365_loss: 8.6438e-09
Epoch 1/1
64/64 [==============================] - 0s 748us/step - loss: 4.8418e-08 - dense_361_loss: 7.7120e-09 - dense_362_loss: 1.4660e-08 - dense_363_loss: 8.4330e-09 - dense_364_loss: 8.9920e-09 - dense_365_loss: 8.6208e-09
Epoch 1/1
64/64 [==============================] - 0s 670us/step - loss: 4.8328e-08 - dense_36

64/64 [==============================] - 0s 935us/step - loss: 4.3751e-08 - dense_361_loss: 6.9510e-09 - dense_362_loss: 1.4314e-08 - dense_363_loss: 7.3295e-09 - dense_364_loss: 7.7148e-09 - dense_365_loss: 7.4415e-09
Epoch 1/1
64/64 [==============================] - 0s 701us/step - loss: 4.3704e-08 - dense_361_loss: 6.9431e-09 - dense_362_loss: 1.4310e-08 - dense_363_loss: 7.3184e-09 - dense_364_loss: 7.6974e-09 - dense_365_loss: 7.4352e-09
Epoch 1/1
64/64 [==============================] - 0s 764us/step - loss: 4.3655e-08 - dense_361_loss: 6.9344e-09 - dense_362_loss: 1.4304e-08 - dense_363_loss: 7.3050e-09 - dense_364_loss: 7.6806e-09 - dense_365_loss: 7.4306e-09
Epoch 1/1
64/64 [==============================] - 0s 841us/step - loss: 4.3607e-08 - dense_361_loss: 6.9274e-09 - dense_362_loss: 1.4299e-08 - dense_363_loss: 7.2917e-09 - dense_364_loss: 7.6636e-09 - dense_365_loss: 7.4260e-09
Epoch 1/1
64/64 [==============================] - 0s 764us/step - loss: 4.3559e-08 - dense_36

64/64 [==============================] - 0s 857us/step - loss: 4.1068e-08 - dense_361_loss: 6.4954e-09 - dense_362_loss: 1.4030e-08 - dense_363_loss: 6.6626e-09 - dense_364_loss: 6.7392e-09 - dense_365_loss: 7.1401e-09
Epoch 1/1
64/64 [==============================] - 0s 764us/step - loss: 4.1035e-08 - dense_361_loss: 6.4901e-09 - dense_362_loss: 1.4027e-08 - dense_363_loss: 6.6538e-09 - dense_364_loss: 6.7272e-09 - dense_365_loss: 7.1365e-09
Epoch 1/1
64/64 [==============================] - 0s 904us/step - loss: 4.1004e-08 - dense_361_loss: 6.4850e-09 - dense_362_loss: 1.4024e-08 - dense_363_loss: 6.6450e-09 - dense_364_loss: 6.7170e-09 - dense_365_loss: 7.1328e-09
Epoch 1/1
64/64 [==============================] - 0s 1ms/step - loss: 4.0969e-08 - dense_361_loss: 6.4797e-09 - dense_362_loss: 1.4021e-08 - dense_363_loss: 6.6362e-09 - dense_364_loss: 6.7045e-09 - dense_365_loss: 7.1276e-09
Epoch 1/1
64/64 [==============================] - 0s 888us/step - loss: 4.0938e-08 - dense_361_

In [78]:
def preds_multi_task2(model,xtest):
    pred_final=[]
    preds = model.predict([xtest,xtest,xtest,xtest,xtest], verbose=False)
    
    for i in range(len(preds[0])):
        pred_final.append([preds[0][i],preds[1][i],preds[2][i],preds[3][i],preds[4][i]])
    
            
    preds_all=[list_cat[np.argmax(l)] for l in pred_final]
    return preds_all

In [340]:
preds_all= preds_multi_task2(model_multi_task,xtest)

In [341]:
ytest[1]

array([1, 0, 0, 0, 0], dtype=int64)

In [342]:
preds_all

['HYPER',
 'COH',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'RANDOM',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'SYN',
 'HYPER',
 'RANDOM',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'SYN',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'RANDOM',
 'HYPER',
 'SYN',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'COH',
 'HYPER',
 'HYPER',
 'HYPER',
 'COH',
 'HYPER',
 'HYPER',
 'HYPER',
 'SYN',
 'HYPER',
 'RANDOM',
 'RANDOM',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'SYN',
 'SYN',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'RANDOM',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'RANDOM',
 'RANDOM',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'RANDOM',
 'HYPER',
 'SYN',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'HYPER',
 'RANDOM',
 'HYPER',
 'HYPER',
 'COH',
 'HYPER',
 'RANDOM',
 'HYPER',
 'HY

In [323]:
pred_final

[[array([0.00257877], dtype=float32),
  array([0.99953943], dtype=float32),
  array([0.01068187], dtype=float32),
  array([0.00140557], dtype=float32),
  array([0.04716048], dtype=float32)],
 [array([0.00339231], dtype=float32),
  array([0.00137678], dtype=float32),
  array([0.00034973], dtype=float32),
  array([0.00241414], dtype=float32),
  array([0.00043327], dtype=float32)],
 [array([0.0026511], dtype=float32),
  array([0.49836594], dtype=float32),
  array([0.0003745], dtype=float32),
  array([0.01069084], dtype=float32),
  array([0.00517768], dtype=float32)],
 [array([0.00270951], dtype=float32),
  array([0.84819293], dtype=float32),
  array([0.00069404], dtype=float32),
  array([0.00036824], dtype=float32),
  array([0.9194171], dtype=float32)],
 [array([0.00255468], dtype=float32),
  array([0.14676791], dtype=float32),
  array([0.00037804], dtype=float32),
  array([0.00241184], dtype=float32),
  array([0.0004698], dtype=float32)],
 [array([0.00258639], dtype=float32),
  array([0.

In [343]:
df_metrics=create_metrics(ytest,preds_all)

['COH', 'HYPER', 'MERO', 'RANDOM', 'SYN']


In [344]:
df_metrics

,F1,ACC
COH,0.185281,0.880402
HYPER,0.397360,0.454499
MERO,0.058824,0.943060
RANDOM,0.241935,0.498221
SYN,0.119437,0.801347


In [215]:
def preds_multi_task(model,xtest):
    preds = model_multi_task.predict([xtest,xtest,xtest,xtest], verbose=False)
    seuil=0.5
    th_coh=seuil
    th_hyper=seuil
    th_mero=seuil
    th_syn=seuil
    pred_coh=(preds[0]>th_coh).astype(int)
    pred_hyper=(preds[1]>th_hyper).astype(int)
    pred_mero=(preds[2]>th_mero).astype(int)
    pred_syn=(preds[3]>th_syn).astype(int)
    pred_final=[]
    for i in range(len(pred_coh)):
        if pred_coh[i]+pred_hyper[i]+pred_mero[i]+pred_syn[i]==0 :
                pred_final.append([pred_coh[i],pred_hyper[i],pred_mero[i],1,pred_syn[i]])
        else :
            pred_final.append([pred_coh[i],pred_hyper[i],pred_mero[i],0,pred_syn[i]])
    return pred_final

In [217]:
pred_final = preds_multi_task(model_multi_task,xtest)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 5 array(s), but instead got the following list of 4 arrays: [array([[ 0.27805  , -0.10738  , -0.3175   , ..., -0.2996   ,  0.53413  ,
         0.76344  ],
       [ 0.31463  , -0.20365  , -0.12585  , ...,  0.3669   , -0.46117  ,
         0.35392  ],
       [ 0....

In [162]:
preds_all=[list_cat[np.argmax(l)] for l in pred_final]

In [163]:
preds_all

['HYPER',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'RANDOM',
 'SYN',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'SYN',
 'RANDOM',
 'SYN',
 'SYN',
 'RANDOM',
 'RANDOM',
 'SYN',
 'COH',
 'HYPER',
 'RANDOM',
 'HYPER',
 'HYPER',
 'SYN',
 'HYPER',
 'RANDOM',
 'COH',
 'RANDOM',
 'RANDOM',
 'SYN',
 'HYPER',
 'HYPER',
 'RANDOM',
 'HYPER',
 'SYN',
 'HYPER',
 'RANDOM',
 'MERO',
 'RANDOM',
 'HYPER',
 'COH',
 'RANDOM',
 'SYN',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'SYN',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'SYN',
 'RANDOM',
 'RANDOM',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'COH',
 'RANDOM',
 'RANDOM',
 'HYPER',
 'HYPER',
 'RANDOM',
 'RANDOM',
 'HYPER',
 'HYPER',
 'RANDOM',
 'HYPER',
 'COH',
 'SYN',
 'RANDOM',
 'COH',
 'RANDOM',
 'SYN',
 'RANDOM',
 'SYN',
 'HYPER',
 'RANDOM',
 'RANDOM',
 'SYN',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'RANDOM',
 'RANDOM',
 'RANDOM',
 'RANDOM',
 'HYPER',
 'HYPER',
 'RANDOM',
 'RANDOM',
 'RANDOM',
 'SYN',
 'RANDOM',
 'RANDOM',
 'RANDOM',
 'HYPER',
 'RANDOM',
 'RAND

In [202]:
    
df_metrics=create_metrics(ytest,pred_final)

['COH', 'HYPER', 'MERO', 'RANDOM', 'SYN']


In [328]:
df_metrics['F1']['COH']

0.16885389326334208

In [157]:
df_NN_Multi

,F1,ACC
COH,0.414045,0.898195
HYPER,0.474820,0.768048
MERO,0.510204,0.951195
RANDOM,0.726097,0.720005
SYN,0.304790,0.817361


In [97]:
ytest

array([[0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       ...,
       [0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0]], dtype=int64)

In [ ]:
def pred_multi_task_random(model,xtest):

    preds = model.predict([xtest,xtest,xtest,xtest], verbose=False)
    preds_all=[list_cat[np.argmax(l)] for l in preds]
    return preds_all